In [ ]:
!pip install --q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login

In [ ]:
!git clone https://github.com/bigcode-project/bigcode-dataset
%cd /content/bigcode-dataset/near_deduplication

Cloning into 'bigcode-dataset'...
remote: Enumerating objects: 789, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 789 (delta 155), reused 212 (delta 123), pack-reused 511
Receiving objects: 100% (789/789), 3.04 MiB | 22.53 MiB/s, done.
Resolving deltas: 100% (458/458), done.
/content/bigcode-dataset/near_deduplication


In [ ]:
!pip install --q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict, load_from_disk

In [ ]:
def add_text(sample):
  return {
      "text": sample["bot_name"] + "\n" + sample["personalities"] + "\n" + sample["categories"]
  }

def prepare_dataset(dataset_path):
  ds = load_dataset(dataset_path)
  joined_ds = concatenate_datasets(ds.values())
  samples = joined_ds.to_list()
  deduplicated_samples = list({v['bot_name']: v for v in samples}.values())
  deduplicated_dataset = Dataset.from_list(deduplicated_samples)
  deduplicated_dataset = deduplicated_dataset.map(add_text)
  name_deduped_path = dataset_path + "-name-deduped"
  deduplicated_dataset.push_to_hub(name_deduped_path, private=True)
  !python minhash_deduplication.py --dataset $name_deduped_path \
    --split train \
    --column text \
    --cache-dir .cache \
    --min-ngram-size 1 \
    --ngram-size 1 \
    --threshold 0.65
  deduplicated_dataset = load_from_disk("output/deduplicated")
  deduplicated_dataset = deduplicated_dataset.remove_columns("text")
  deduplicated_dataset.push_to_hub(dataset_path + "-prepared")
  print(deduplicated_dataset)
  return deduplicated_dataset

In [ ]:
prepared_dataset = prepare_dataset("AlekseyKorshuk/character-profiles-romance")

In [ ]:
prepared_dataset = prepare_dataset("AlekseyKorshuk/character-profiles-friendly")

In [ ]:
prepared_dataset = prepare_dataset("AlekseyKorshuk/character-profiles-fight")